# 1. Medallion Architecture

## 1.1. The Story: Building the Data Factory

Your e-commerce platform now has Delta Lake running. But there's chaos:
- 15 different notebooks, each loading data differently
- No naming conventions - `customers_v2_final_FINAL.csv` anyone?
- Data Scientists use Bronze data directly (with all the garbage)
- Finance dashboard shows different numbers than Marketing

**The CTO asks:** "Can we standardize this before we hire more engineers?"

**The Answer:** Medallion Architecture - a proven pattern used by Netflix, Uber, and thousands of companies.

---

## 1.2. Why Medallion? (The Engineering Case)

### The Pattern
```
┌─────────────┐      ┌─────────────┐      ┌─────────────┐
│   BRONZE    │ ──▶  │   SILVER    │ ──▶  │    GOLD     │
│   (Raw)     │      │  (Cleaned)  │      │ (Business)  │
├─────────────┤      ├─────────────┤      ├─────────────┤
│ - As-is     │      │ - Validated │      │ - Star schema│
│ - Append    │      │ - Dedupe    │      │ - Aggregates │
│ - All types │      │ - Typed     │      │ - SCD applied│
└─────────────┘      └─────────────┘      └─────────────┘
     Ingest              Transform           Serve
```

### Why Three Layers? (Not Two, Not Five)

| Layers | Issue |
|--------|---------|
| **1 layer** | No separation of concerns, risky transformations on raw data |
| **2 layers** | "Where do we put validation?" - unclear ownership |
| **3 layers** | Clear responsibilities, testable boundaries |
| **5+ layers** | Over-engineering, increased latency, maintenance nightmare |

*Three layers hit the sweet spot for most organizations.*

### Layer Responsibilities

| Layer | Owner | SLA | Access |
|-------|-------|-----|--------|
| **Bronze** | Data Engineering | 15 min from source | Engineers only |
| **Silver** | Data Engineering | 1 hour from Bronze | Engineers + DS |
| **Gold** | Analytics Engineering | 4 hours from Silver | Everyone |

### Cost Implications

| Decision | Impact |
|----------|-------------|
| Keep Bronze forever | Storage grows linearly (plan retention!) |
| Skip Silver layer | Technical debt accumulates, bugs in Gold |
| Over-aggregate Gold | Every new question = new pipeline |

**Recommendation:** 
- Bronze: 90 days retention (or regulatory requirement)
- Silver: Indefinite (your master data)
- Gold: Depends on use case (often regenerated)

---

## 1.3. What You'll Build

| Layer | Tables | Transformations |
|-------|--------|-----------------|
| **Bronze** | customers, orders, products | Raw ingestion, metadata |
| **Silver** | customers, orders, products | Validation, dedup, typing |
| **Gold** | fact_sales, dim_customer, dim_product, dim_date | Star schema, SCD |

---

## 1.4. The Medallion Pattern

```
┌─────────────────┐      ┌─────────────────┐      ┌─────────────────┐
│     BRONZE      │ ──▶  │     SILVER      │ ──▶  │      GOLD       │
│     (Raw)       │      │   (Cleaned)     │      │   (Business)    │
├─────────────────┤      ├─────────────────┤      ├─────────────────┤
│ • Raw Data      │      │ • Validation    │      │ • Star schema   │
│ • Append-only   │      │ • Deduplication │      │ • Aggregates    │
│ • All types     │      │ • Typing        │      │ • SCD applied   │
│ • +Metadata     │      │ • Normalization │      │ • Business ready│
└─────────────────┘      └─────────────────┘      └─────────────────┘
      INGEST                TRANSFORM               SERVE
```

## 1.5. Why Three Layers? (Not Two, Not Five)

| Layers | Issue |
|--------|---------|
| **1 layer** | No separation, risky transformations on raw data |
| **2 layers** | "Where to put validation?" - unclear responsibility |
| **3 layers** | Clear responsibilities, testable boundaries |

## 1.6. Layer Responsibilities

| Layer | Owner | SLA | Access | Retention |
|-------|-------|-----|--------|----------|
| **Bronze** | Data Engineering | 15 min from source | Engineers only | 90 days (or regulatory) |
| **Silver** | Data Engineering | 1h from Bronze | Engineers + DS | Indefinite (master data) |
| **Gold** | Analytics Engineering | 4h from Silver | Everyone | Depends on use case |

### Bronze Layer Details
- **Goal:** Preserve raw data without modification
- **Transformations:** Only adding metadata (`_metadata.file_path`, `ingestion_ts`, `load_ts`)
- **Format:** Delta Table (Streaming Table in Lakeflow)
- **Schema:** Often inferred, can evolve

### Silver Layer Details
- **Goal:** Cleaned, validated, normalized data
- **Transformations:** Validation, deduplication, typing, derived calculations
- **Format:** Delta Table with constraints
- **Schema:** Strictly defined, versioned

### Gold Layer Details
- **Goal:** Data ready for business consumption
- **Transformations:** Star schema, aggregations, SCD
- **Format:** Materialized Views or Delta Tables
- **Schema:** Optimized for analytical queries

## 1.7. What We'll Build

```
┌──────────────┐     ┌──────────────┐     ┌──────────────────┐
│   BRONZE     │     │    SILVER    │     │      GOLD        │
├──────────────┤     ├──────────────┤     ├──────────────────┤
│ customers    │ ──▶ │ customers    │ ──▶ │ dim_customer     │
│ orders       │ ──▶ │ orders       │ ──▶ │ fact_sales       │
│ products     │ ──▶ │ products     │ ──▶ │ dim_product      │
│              │     │              │     │ dim_date         │
│              │     │              │     │ dim_payment      │
│              │     │              │     │ dim_store        │
└──────────────┘     └──────────────┘     └──────────────────┘
```

---

# 2. SCD Type 1 & Type 2

## 2.1. Goal

Understanding **Slowly Changing Dimensions** - how to track changes in dimensional data.

## 2.2. What is SCD (Slowly Changing Dimension)?

**Slowly Changing Dimension (SCD)** is a data warehousing concept describing how to handle changes in dimensional data.

### Example: Customer address change

Customer John Smith moved from Warsaw to Krakow. What do we do?

| Type | Strategy | Result |
|-----|-----------|----------|
| **SCD Type 0** | Retain original | Always Warsaw |
| **SCD Type 1** | Overwrite | Only Krakow |
| **SCD Type 2** | Track history | Both records with dates |
| **SCD Type 3** | Add column | `current_city=Krakow`, `previous_city=Warsaw` |

## 2.3. SCD Type 1 - Overwrite

### Characteristics:
- **Simplest** approach
- **No history** - old values are overwritten
- **Use cases:** Error corrections, data that don't require history

### Before changes:
```
| customer_id | name         | city      |
|-------------|--------------|----------|
| C001        | John Smith | Warsaw |
```

### After changes:
```
| customer_id | name         | city    |
|-------------|--------------|--------|
| C001        | John Smith | Krakow |
```

### Implementation in SQL (MERGE):
```sql
MERGE INTO dim_customer t
USING source_customers s
ON t.customer_id = s.customer_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
```

## 2.4. SCD Type 2 - Track History

### Characteristics:
- **Full history** of changes
- **Additional columns:** `__START_AT`, `__END_AT` (or `valid_from`, `valid_to`)
- **Use cases:** Audit, historical analysis, compliance

### Before changes:
```
| customer_id | name         | city      | __START_AT          | __END_AT |
|-------------|--------------|-----------|---------------------|----------|
| C001        | John Smith | Warsaw  | 2024-01-01 00:00:00 | NULL     |
```

### After changes:
```
| customer_id | name         | city      | __START_AT          | __END_AT            |
|-------------|--------------|-----------|---------------------|---------------------|
| C001        | John Smith | Warsaw  | 2024-01-01 00:00:00 | 2024-06-15 10:30:00 |
| C001        | John Smith | Krakow    | 2024-06-15 10:30:00 | NULL                |
```

### Queries:
```sql
-- Current state (snapshot)
SELECT * FROM silver_customers WHERE __END_AT IS NULL;

-- State at specified moment (business time travel)
SELECT * FROM silver_customers 
WHERE '2024-03-01' BETWEEN __START_AT AND COALESCE(__END_AT, '9999-12-31');
```

## 2.5. SCD Type 2 in Lakeflow - AUTO CDC

Databricks Lakeflow offers built-in support for SCD Type 2 through `AUTO CDC INTO`:

```sql
-- Creating target table SCD2
CREATE OR REFRESH STREAMING TABLE silver_customers (
  customer_id        STRING,
  first_name         STRING,
  last_name          STRING,
  city               STRING,
  -- SCD2 columns added automatically:
  __START_AT         TIMESTAMP,
  __END_AT           TIMESTAMP
);

-- Flow with AUTO CDC for SCD2
CREATE FLOW silver_customers_scd2_flow
AS AUTO CDC INTO silver_customers
FROM STREAM bronze_customers
KEYS (customer_id)         -- Business key
SEQUENCE BY ingestion_ts   -- Column determining the order
STORED AS SCD TYPE 2;      -- SCD Type
```

### Key elements:
- **KEYS** - columns identifying the business record
- **SEQUENCE BY** - column determining the order of changes
- **STORED AS SCD TYPE 1|2** - SCD type

### SCD Type 1 in Lakeflow:
```sql
CREATE FLOW silver_products_scd1_flow
AS AUTO CDC INTO silver_products
FROM bronze_products
KEYS (product_id)
SEQUENCE BY ingestion_ts
STORED AS SCD TYPE 1;  -- Overwrite without history
```

---

# 3. Lakeflow Pipelines - Declarations

## 3.1. Goal

Learning the syntax **Spark Declarative Pipelines** in SQL and PySpark - declarative way of defining data pipelines.

## 3.2. What is Lakeflow?

**Lakeflow** (formerly Delta Live Tables - DLT) is a Databricks framework for building declarative data pipelines.

### Declarative vs Imperative

| Approach | Example | Characteristics |
|-----------|----------|----------------|
| **Imperative** | `df.write.mode("overwrite")...` | You describe HOW |
| **Declarative** | `CREATE TABLE AS SELECT...` | You describe WHAT |

### Lakeflow Benefits:
- ✅ **Automatic dependency management** between tables
- ✅ **Built-in data quality** - constraints with actions
- ✅ **Unified batch/streaming** - same code
- ✅ **Automatic recovery** after failures
- ✅ **Lineage and monitoring** out-of-the-box
- ✅ **Incremental processing** - only new data

## 3.3. Table Types in Lakeflow

| Type | Usage | Characteristics |
|-----|--------|----------------|
| **STREAMING TABLE** | Data source, append-only | Processes only new data |
| **MATERIALIZED VIEW** | Aggregations, transformations | Always full recomputation |
| **VIEW** | Intermediate logic | Not materialized |

## 3.4. SQL: STREAMING TABLE (Bronze)

```sql
CREATE OR REFRESH STREAMING TABLE bronze_customers
AS
SELECT
  customer_id,
  first_name,
  last_name,
  email,
  city,
  _metadata.file_path                AS source_file_path,
  _metadata.file_modification_time   AS ingestion_ts,
  current_timestamp()                AS load_ts
FROM STREAM read_files(
  '${customer_path}',
  format           => 'csv',
  header           => true,
  inferColumnTypes => true
);
```

### Key elements:
- **`STREAM read_files(...)`** - reads files in streaming mode
- **`_metadata`** - source file metadata
- **`${variable}`** - pipeline parameters

## 3.5. SQL: STREAMING TABLE with CONSTRAINTS (Silver)

```sql
CREATE OR REFRESH STREAMING TABLE silver_orders
(
  CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL)
    ON VIOLATION DROP ROW,
  CONSTRAINT valid_customer EXPECT (customer_id IS NOT NULL)
    ON VIOLATION DROP ROW,
  CONSTRAINT valid_quantity EXPECT (quantity > 0)
    ON VIOLATION DROP ROW,
  CONSTRAINT valid_price EXPECT (unit_price >= 0)
    ON VIOLATION FAIL UPDATE
)
AS
SELECT
  order_id,
  customer_id,
  product_id,
  CAST(order_datetime AS TIMESTAMP) AS order_ts,
  quantity,
  unit_price,
  (quantity * unit_price) AS gross_amount
FROM STREAM(bronze_orders);
```

### Actions for CONSTRAINTS:
- **`DROP ROW`** - remove invalid record
- **`FAIL UPDATE`** - fail the pipeline
- No action - log only

## 3.6. SQL: MATERIALIZED VIEW (Gold)

```sql
-- Dimension - current snapshot from SCD2
CREATE OR REFRESH MATERIALIZED VIEW dim_customer
AS
SELECT
  customer_id,
  first_name,
  last_name,
  email,
  city,
  customer_segment
FROM silver_customers
WHERE __END_AT IS NULL;

-- Date Dimension
CREATE OR REFRESH MATERIALIZED VIEW dim_date
AS
SELECT DISTINCT
  CAST(date_format(order_date, 'yyyyMMdd') AS INT) AS date_key,
  order_date AS date,
  year(order_date) AS year,
  quarter(order_date) AS quarter,
  month(order_date) AS month
FROM silver_orders;

-- Fact - streaming from Silver
CREATE OR REFRESH STREAMING TABLE fact_sales
AS
SELECT
  order_id,
  customer_id,
  product_id,
  order_date_key,
  quantity,
  gross_amount,
  net_amount
FROM STREAM(silver_orders);
```

## 3.7. What is FLOW?

**FLOW** is a Lakeflow mechanism that allows:
1. **Separation of table definition from data source**
2. **Multiple sources to one table** (e.g. backfill + streaming)
3. **CDC (Change Data Capture)** with automatic SCD

### FLOW Anatomy:

```sql
-- 1. We define empty target table
CREATE OR REFRESH STREAMING TABLE bronze_orders;

-- 2. We define FLOW(s) which populate it
CREATE FLOW flow_name
AS INSERT INTO target_table BY NAME
SELECT ... FROM source;
```

## 3.8. FLOW: Backfill + Streaming Pattern

```sql
-- Target table
CREATE OR REFRESH STREAMING TABLE bronze_orders;

-- FLOW 1: One-time backfill
CREATE FLOW bronze_orders_backfill
AS 
INSERT INTO ONCE bronze_orders BY NAME
SELECT
  order_id,
  customer_id,
  product_id,
  order_datetime,
  'batch' AS source_system,
  _metadata.file_path AS source_file_path,
  current_timestamp() AS load_ts
FROM read_files(
  '${order_path}/orders_batch.json',
  format => 'json'
);

-- FLOW 2: Continuous streaming
CREATE FLOW bronze_orders_stream
AS 
INSERT INTO bronze_orders BY NAME
SELECT
  order_id,
  customer_id,
  'stream' AS source_system,
  _metadata.file_path AS source_file_path,
  current_timestamp() AS load_ts
FROM STREAM read_files(
  '${order_path}/stream/orders_stream_*.json',
  format => 'json'
);
```

### Key elements:
- **`INSERT INTO ONCE`** - execute once (backfill)
- **`INSERT INTO`** - continuous insertion (streaming)
- **`BY NAME`** - column mapping by name

## 3.9. FLOW: AUTO CDC for SCD Type 2

```sql
-- SCD2 Table with schema
CREATE OR REFRESH STREAMING TABLE silver_customers (
  customer_id        STRING,
  first_name         STRING,
  last_name          STRING,
  email              STRING,
  city               STRING,
  __START_AT         TIMESTAMP,
  __END_AT           TIMESTAMP
);

-- AUTO CDC Flow
CREATE FLOW silver_customers_scd2_flow
AS AUTO CDC INTO silver_customers
FROM STREAM bronze_customers
KEYS (customer_id)
SEQUENCE BY ingestion_ts
STORED AS SCD TYPE 2;
```

### What does AUTO CDC do?
1. Compares new records with existing by `KEYS`
2. Detects changes in columns
3. For SCD2: closes old record (`__END_AT`), inserts new
4. For SCD1: overwrites existing record

## 3.10. PySpark Declarations

```python
from pyspark import pipelines as dp
from pyspark.sql.functions import *

# STREAMING TABLE
@dp.table(
    name="bronze_customers",
    comment="Raw customers from CSV"
)
def bronze_customers():
    return (
        spark.readStream
            .format("cloudFiles")
            .option("cloudFiles.format", "csv")
            .option("header", "true")
            .load(spark.conf.get("customer_path"))
            .select(
                "*",
                "_metadata.file_path".alias("source_file_path"),
                current_timestamp().alias("load_ts")
            )
    )

# MATERIALIZED VIEW
@dp.table(name="dim_customer")
def dim_customer():
    return (
        spark.read.table("silver_customers")
            .filter(col("__END_AT").isNull())
            .select("customer_id", "first_name", "last_name")
    )
```

## 3.11. PySpark: Expectations (Constraints)

```python
@dp.table(name="silver_orders")
@dp.expect_or_drop("valid_order_id", "order_id IS NOT NULL")
@dp.expect_or_drop("valid_customer", "customer_id IS NOT NULL")
@dp.expect_or_drop("valid_quantity", "quantity > 0")
@dp.expect_or_fail("valid_price", "unit_price >= 0")
def silver_orders():
    return (
        spark.readStream.table("bronze_orders")
            .select(
                "order_id",
                "customer_id",
                col("order_datetime").cast("timestamp").alias("order_ts"),
                (col("quantity") * col("unit_price")).alias("gross_amount")
            )
    )
```

### Expectations Decorators:
- **`@dp.expect`** - log only
- **`@dp.expect_or_drop`** - drop the record
- **`@dp.expect_or_fail`** - fail the pipeline

## 3.12. PySpark: apply_changes (CDC)

```python
from pyspark import pipelines as dp

# Define the target table
dp.create_streaming_table(
    name="silver_customers",
    schema="""
        customer_id STRING,
        first_name STRING,
        city STRING,
        __START_AT TIMESTAMP,
        __END_AT TIMESTAMP
    """
)

# Define the CDC flow
dp.create_auto_cdc_flow(
    target="silver_customers",
    source="bronze_customers",
    keys=["customer_id"],
    sequence_by="ingestion_ts",
    stored_as_scd_type=2  # or 1
)
```

---

# 4. DEMO - Building the Pipeline

## 4.1. Goal

Practical construction of a Lakeflow pipeline step-by-step in Databricks.

## 4.2. SQL Files Overview

Our pipeline consists of SQL files in folder `lakeflow/`:

```
lakeflow/
├── 01_bronze/
│   ├── bronze_customers.sql
│   ├── bronze_orders.sql
│   └── bronze_products.sql
├── 02_silver/
│   ├── silver_customers.sql
│   ├── silver_orders.sql
│   └── silver_products.sql
└── 03_gold/
    ├── dim_customer.sql
    ├── dim_date.sql
    ├── dim_payment_method.sql
    ├── dim_product.sql
    ├── dim_store.sql
    └── fact_sales.sql
```

## 4.3. Step 1: Upload SQL Files

### Option A: Via UI
1. Databricks → Workspace → Users → (your folder)
2. Create folder `lakeflow_pipeline`
3. Upload SQL files from folder `lakeflow/`

### Option B: Via Repos
1. Databricks → Repos → Add Repo
2. Clone training repository

## 4.4. Step 2: Create Pipeline

1. Go to **Workflows** → **Delta Live Tables**
2. Click **Create Pipeline**
3. Fill in details:
   - **Pipeline Name:** `lakeflow_pipeline_<your_name>`
   - **Product Edition:** `Advanced`
   - **Pipeline Mode:** `Triggered` (for dev)
   - **Source Code:** Select folder with SQL files (from Step 1)
   - **Storage Location:** `dbfs:/pipelines/lakeflow_<your_name>`
   - **Target Schema:** `<your_schema>_lakeflow`

## 4.5. Step 3: Configure Variables

In **Configuration** → **Add configuration**:

| Key | Value |
|-----|-------|
| `customer_path` | `/Volumes/training_catalog/default/kion_datasets/customers` |
| `order_path` | `/Volumes/training_catalog/default/kion_datasets/orders` |
| `product_path` | `/Volumes/training_catalog/default/kion_datasets/products` |

## 4.6. Step 4: Validate & Run

1. Click **Validate** - check syntax and DAG
2. Click **Start** - run pipeline
3. Observe execution:
   - **Initializing** → **Setting up tables** → **Running** → **Completed**

## 4.7. Step 5: Verify Results

In [ ]:
# Run in Databricks after pipeline execution

catalog_name = spark.conf.get("catalog_name")
user_schema = spark.conf.get("user_schema")

print("=" * 50)
print("Created tables:")
print("=" * 50)
display(spark.sql(f"SHOW TABLES IN {catalog_name}.{user_schema}_lakeflow"))


In [ ]:
# Check Bronze - raw data with metadata
display(spark.sql(f"""
    SELECT * 
    FROM {catalog_name}.{user_schema}_lakeflow.bronze_customers 
    LIMIT 5
"""))


In [ ]:
# Check Silver customers - SCD Type 2
display(spark.sql(f"""
    SELECT 
        customer_id, first_name, last_name, city,
        __START_AT, __END_AT
    FROM {catalog_name}.{user_schema}_lakeflow.silver_customers 
    ORDER BY customer_id, __START_AT
    LIMIT 10
"""))


In [ ]:
# Check Gold - dim_customer (current snapshot)
display(spark.sql(f"""
    SELECT * 
    FROM {catalog_name}.{user_schema}_lakeflow.dim_customer 
    LIMIT 5
"""))


In [ ]:
# Check fact_sales with joins to dimensions
display(spark.sql(f"""
    SELECT 
        f.order_id,
        c.first_name || ' ' || c.last_name AS customer_name,
        p.product_name,
        d.date,
        f.quantity,
        f.net_amount
    FROM {catalog_name}.{user_schema}_lakeflow.fact_sales f
    LEFT JOIN {catalog_name}.{user_schema}_lakeflow.dim_customer c ON f.customer_id = c.customer_id
    LEFT JOIN {catalog_name}.{user_schema}_lakeflow.dim_product p ON f.product_id = p.product_id
    LEFT JOIN {catalog_name}.{user_schema}_lakeflow.dim_date d ON f.order_date_key = d.date_key
    LIMIT 10
"""))


## 4.8. Step 6: Test Incremental Processing

1. Add new file to folder orders/stream/
2. Run pipeline again
3. Check Event Log - should process only new files

## 4.9. Step 7: Test SCD Type 2

In [ ]:
# Znajdź klientów z historią zmian
display(spark.sql(f"""
    SELECT 
        customer_id, first_name, city,
        __START_AT, __END_AT,
        CASE WHEN __END_AT IS NULL THEN 'Current' ELSE 'Historical' END AS status
    FROM {catalog_name}.{user_schema}_lakeflow.silver_customers
    WHERE customer_id IN (
        SELECT customer_id 
        FROM {catalog_name}.{user_schema}_lakeflow.silver_customers 
        GROUP BY customer_id HAVING COUNT(*) > 1
    )
    ORDER BY customer_id, __START_AT
"""))

## 4.10. Monitoring and Troubleshooting

### Event Log
- Click on table in DAG → processing details
- Metrics: records processed, dropped rows, duration

### Common issues:

| Issue | Cause | Solution |
|---------|-----------|-------------|
| Pipeline hangs | Cluster too small | Increase min workers |
| Missing data | Constraint DROP ROW | Check Data Quality tab |
| Schema mismatch | Schema change | Full refresh |

---

# 5. Summary

## 5.1. Key Takeaways

### Medallion Architecture
- **Bronze:** Raw data + metadata
- **Silver:** Cleaned, validated
- **Gold:** Business-ready, star schema

### SCD Types
- **SCD Type 1:** Overwrite - no history
- **SCD Type 2:** Track history - `__START_AT`, `__END_AT`

### Lakeflow Declarations
- **STREAMING TABLE:** Data append-only
- **MATERIALIZED VIEW:** Aggregations
- **FLOW:** CDC, backfill+streaming

### Best Practices
1. Constraints with `DROP ROW` for quality
2. Backfill (ONCE) + streaming FLOWs
3. SCD Type 2 for dimensions with history
4. STREAMING TABLE for facts

## 5.2. Resources

- [Databricks Lakeflow Docs](https://docs.databricks.com/en/delta-live-tables/index.html)
- [Medallion Architecture](https://www.databricks.com/glossary/medallion-architecture)
- [SCD with Lakeflow](https://docs.databricks.com/en/delta-live-tables/cdc.html)

---

## 5.3. Next Steps

1. **Notebook 06:** Orchestration - Jobs & Workflows
2. **Notebook 07:** Unity Catalog & Governance

## Whatntext and Requirements

**Requirements:**
- Databricks Runtime 16.4 LTS or newer (recommended: 17.3 LTS)
- Unity Catalog enabled
- Access to `/Volumes/` or DBFS for datasets
- `00_setup.ipynb` notebook executed (for user isolation)

**Prerequisite:**
- Whatmpleted training (or familiarity with) notebooks:
  - `01_databricks_lakehouse_intro.ipynb`
  - `02_data_import_exploration.ipynb`
  - `03_delta_lake_operations.ipynb`
  - `04_optimization_best_practices.ipynb`
  - `05_lakeflow_connection.ipynb`

## Theoretical Introduction - Medallion Architecture

### What is Medallion Architecture?

**Medallion Architecture** is a design pattern for organizing data in a Data Lakehouse.

It divides data into three layers based on **quality level and purpose**:

```
📥 RAW DATA          🔄 CLEAN DATA           📊 BUSINESS DATA
   (Sources)            (Validated)              (Analytics)
      ↓                     ↓                        ↓
┌─────────────┐      ┌─────────────┐         ┌─────────────┐
│   BRONZE    │ ──▶  │   SILVER    │ ──▶     │    GOLD     │
│  Raw Data   │      │   Clean     │         │  Business   │
│  "As-Is"    │      │  Validated  │         │   Ready     │
└─────────────┘      └─────────────┘         └─────────────┘
```

---

### Bronze Layer - Raw Data

**Purpose:** Landing zone for raw data without transformations.

**Key Characteristics:**
- **Raw data "as-is"** - no business logic changes
- **Append-only** - data is only added, never modified
- **Multi-format support** - JSON, CSV, Parquet, XML
- **Schema-on-read** - flexible schema
- **Audit metadata** - source, timestamp, file path

**Metadata Whatlumns (Lakeflow standard):**
```python
source_file_path     # Source file path
ingestion_ts         # File processing timestamp  
load_ts              # Load to Bronze timestamp
source_system        # Source system (batch/stream)
```

**Use Cases:**
- ✅ Backup & recovery of raw data
- ✅ Reprocessing pipelines from scratch
- ✅ Whatmpliance & audit (full history)
- ✅ Data science exploration

---

### Silver Layer - Clean & Validated

**Purpose:** Cleansing and validation for analytical use.

**Key Characteristics:**
- **Deduplication** - removal of duplicate records
- **Type casting** - correct data types
- **Null handling** - replacement with defaults or Unknown
- **Business rules** - validation and calculated fields
- **SCD (Slowly Changing Dimensions)** - history tracking

**Transformations:**
```python
# Calculated measures (Lakeflow model)
gross_amount = quantity * unit_price
discount_amount = gross_amount * discount_percent / 100
net_amount = gross_amount - discount_amount

# Quality flags
is_return = 1 if quantity < 0 else 0
is_future_dated = 1 if order_date > current_date else 0
is_unknown_customer = 1 if customer_id IS NULL else 0
```

**Whatnstraints (Lakeflow):**
```sql
CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW
CONSTRAINT valid_quantity EXPECT (quantity IS NOT NULL AND quantity <> 0) ON VIOLATION DROP ROW
```

---

### Gold Layer - Business Ready

**Purpose:** Aggregated data ready for Business Intelligence.

**Key Characteristics:**
- **Star Schema** - fact tables + dimensions
- **Pre-calculated aggregates** - daily, monthly, yearly
- **Domain-specific views** - Marketing, Finance, Operations
- **Performance optimized** - indexes, partitioning

**Star Schema Example:**
```
                    ┌─────────────┐
                    │  dim_date   │
                    └──────┬──────┘
                           │
┌─────────────┐    ┌───────┴───────┐    ┌─────────────┐
│dim_customer │────│  fact_sales   │────│ dim_product │
└─────────────┘    └───────┬───────┘    └─────────────┘
                           │
              ┌────────────┴────────────┐
              │                         │
       ┌──────┴──────┐         ┌────────┴────────┐
       │  dim_store  │         │dim_payment_method│
       └─────────────┘         └─────────────────┘
```

---

### ETL vs ELT - Paradigm Shift

**Traditional ETL (Extract-Transform-Load):**
```
Source → Transform (ETL Server) → Data Warehouse
         ⚠️  Expensive compute
         ⚠️  Limited scalability
```

**Modern ELT (Extract-Load-Transform):**
```
Source → Load (Data Lake) → Transform (Spark/DBR)
         ✅ Cheap storage
         ✅ Unlimited scalability
         ✅ Raw data preserved
```

**Medallion = ELT Pattern:**
```
Source → Bronze (Load) → Silver (Transform) → Gold (Aggregate)
```

---

### Production Benefits

**1. Data Recovery:**
```python
# Full pipeline can be reprocessed from Bronze
bronze_data = spark.table("bronze.orders_raw")
# Re-run: Bronze → Silver → Gold
```

**2. Schema Evolution:**
```python
# New columns in source don't break pipeline
# They go to _rescued_data column
```

**3. Whatmpliance & Audit:**
```python
# Full history: who, what, when loaded
# Retention: 3-7 years (legal regulations)
```

**4. Performance Isolation:**
```python
# Bronze: Write-optimized (streaming ingestion)
# Silver: Balanced (MERGE operations)
# Gold: Read-optimized (BI queries)
```

---

### Lakeflow vs Notebook - Whatmparison

| Aspect | Lakeflow (SQL) | Notebook (PySpark) |
|--------|----------------|-------------------|
| **Processing** | Streaming (real-time) | Batch (scheduled) |
| **Syntax** | SQL DDL | Python/PySpark |
| **SCD Type 2** | AUTO CDC | Manual MERGE |
| **Whatnstraints** | EXPECT + ON VIOLATION | .filter() |
| **Orchestration** | Pipelines (auto) | Jobs (manual) |
| **Use Case** | Production | Development/POC |

**Important:** Both approaches create an **identical data model**!

## Per-User Isolation

Run the initialization script for per-user catalog and schema isolation:

In [0]:
%run ../00_setup

## Whatnfiguration

Library imports and environment variable setup:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime, timedelta
import time

# Display user context
display({
    "Catalog": CATALOG,
    "Schema Bronze": BRONZE_SCHEMA,
    "Schema Silver": SILVER_SCHEMA,
    "Schema Gold": GOLD_SCHEMA,
    "User": raw_user,
    "Dataset base": DATASET_BASE_PATH
})

# Set catalog and schema as defaults
spark.sql(f"USE CATALOG {CATALOG}")

### User Whatntext

Displaying current environment configuration and data paths:

In [0]:
spark.sql(f"USE CATALOG {CATALOG}")

**Unity Catalog Whatnfiguration:**

Setting the default catalog for all operations.

## Section 1: Bronze Layer - Raw Data Landing

**Section Objective:** Understand the role of Bronze layer as a landing zone for raw data.

### Bronze Layer - Key Characteristics

**1. Raw Data "As-Is"**
- Data saved without value transformations
- Original format preserved
- Multi-format support (JSON, CSV, Parquet)

**2. Append-Only Pattern**
- Never delete/modify data
- Immutable history
- Time-travel capability

**3. Audit Metadata**
```python
# Audit metadata in Bronze (Lakeflow compliant)
source_file_path     # Source file path
ingestion_ts         # File processing timestamp
load_ts              # Load to Bronze timestamp
```

**4. Schema-on-Read**
- Flexible schema (can change)
- Rescued data column for unknown columns
- Reprocessing capability

### Bronze Tables - Data Model (Lakeflow compliant)

**`bronze_customers`** (CSV):
- `customer_id`, `first_name`, `last_name`, `email`, `phone`
- `city`, `state`, `country`, `registration_date`, `customer_segment`
- Metadata: `source_file_path`, `ingestion_ts`, `load_ts`

**`bronze_orders`** (JSON):
- `order_id`, `customer_id`, `product_id`, `store_id`
- `order_datetime`, `quantity`, `unit_price`, `discount_percent`, `total_amount`
- `payment_method`, `source_system`
- Metadata: `source_file_path`, `ingestion_ts`, `load_ts`

**`bronze_products`** (Parquet):
- `product_id`, `product_name`, `subcategory_code`, `brand`
- `unit_cost`, `list_price`, `weight_kg`, `status`
- Metadata: `source_file_path`, `ingestion_ts`, `load_ts`

### Why is Bronze Important?

**1. Data Recovery**
```python
# We can reprocess pipeline from Bronze
bronze_data = spark.table("bronze.orders_raw")
# Re-run transformations → Silver → Gold
```

**2. Schema Evolution**
```python
# New columns in source don't break pipeline
# They go to _rescued_data
```

**3. Whatmpliance & Audit**
```python
# Full history: who, what, when loaded
# Retention: 3-7 years (legal regulations)
```

**4. Data Science Exploration**
```python
# Analysts can explore raw data
# Create new features from raw data
```

### Example 1.1: Bronze Layer Inspection

**Objective:** Check data in Bronze layer and understand its structure.

### Example 1.1: Ingestion to Bronze Layer (Raw Data)

We load data directly from source files (CSV, JSON, Parquet) into Bronze tables.
This makes the notebook independent from previous steps.

In [0]:
# 1. Load Customers (CSV) - Lakeflow model compliant
customers_path = f"{DATASET_BASE_PATH}/customers/customers.csv"

customers_df = (spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(customers_path)
    # Add metadata columns (as in Lakeflow)
    .withColumn("source_file_path", F.input_file_name())
    .withColumn("ingestion_ts", F.current_timestamp())
    .withColumn("load_ts", F.current_timestamp())
)

display(customers_df.limit(5))

In [0]:
# 3. Load Products (Parquet) - Lakeflow model compliant
products_path = f"{DATASET_BASE_PATH}/products/products.parquet"

products_df = (spark.read
    .format("parquet")
    .load(products_path)
    # Add metadata columns (as in Lakeflow)
    .withColumn("source_file_path", F.input_file_name())
    .withColumn("ingestion_ts", F.current_timestamp())
    .withColumn("load_ts", F.current_timestamp())
)

# Write to Bronze
(products_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{BRONZE_SCHEMA}.products_raw")
)

display({"status": f"✅ Table {BRONZE_SCHEMA}.products_raw created/overwritten"})

In [0]:
# 2. Load Orders (JSON) - Lakeflow model compliant
orders_path = f"{DATASET_BASE_PATH}/orders/orders_batch.json"

orders_df = (spark.read
    .format("json")
    .load(orders_path)
    # Add metadata columns (as in Lakeflow)
    .withColumn("source_system", F.lit("batch"))
    .withColumn("source_file_path", F.input_file_name())
    .withColumn("ingestion_ts", F.current_timestamp())
    .withColumn("load_ts", F.current_timestamp())
)

# Write to Bronze
(orders_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{BRONZE_SCHEMA}.orders_raw")
)

display({"status": f"✅ Table {BRONZE_SCHEMA}.orders_raw created/overwritten"})

In [0]:
# Write Customers to Bronze
(customers_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{BRONZE_SCHEMA}.customers_raw")
)

display({"status": f"✅ Table {BRONZE_SCHEMA}.customers_raw created/overwritten"})

**Created Bronze tables:**

Whatnversion of batch data to raw tables completed successfully.

In [0]:
# Inspekcja Bronze Layer
bronze_tables = ["customers_raw", "orders_raw", "products_raw"]
results = []

for table in bronze_tables:
    full_table = f"{CATALOG}.{BRONZE_SCHEMA}.{table}"
    
    if spark.catalog.tableExists(full_table):
        df = spark.table(full_table)
        results.append({
            "table": table,
            "status": "✅",
            "records": df.count(),
            "columns": len(df.columns)
        })

display(spark.createDataFrame(results))

**Bronze Layer Inspection Summary:**

Bronze Layer contains RAW data without transformations, maintains full history (append-only), and serves as the foundation for further transformations.

## Section 2: Silver Layer - Cleansing & Validation

**Section Objective:** Transform data from Bronze to Silver applying data quality rules.

### Silver Layer - Data Model (Lakeflow compliant)

**`silver_orders`** - Cleaned and calculated measures:
- Keys: `order_id`, `customer_id`, `product_id`, `store_id`
- Dates: `order_ts`, `order_date`, `order_date_key` (INT: yyyyMMdd)
- Original measures: `quantity`, `unit_price`, `discount_percent`
- **Calculated measures:**
  - `gross_amount = quantity * unit_price`
  - `discount_amount = quantity * unit_price * discount_percent / 100`
  - `net_amount = gross_amount - discount_amount`
- `payment_method_code` (standardized)
- **Quality flags:**
  - `is_return` (quantity < 0 OR total_amount < 0)
  - `is_future_dated` (order_date > current_date)
  - `is_unknown_customer`, `is_unknown_product`

**`silver_customers`** (SCD Type 2):
- Fields: `customer_id`, `first_name`, `last_name`, `email`, `phone`, `city`, `state`, `country`
- SCD2 columns: `__START_AT`, `__END_AT` (or `effective_from`, `effective_to`)

**`silver_products`**:
- `product_id`, `product_name`, `subcategory_code`, `brand`
- `unit_cost`, `list_price`, `weight_kg`, `status`
- `is_active`, `is_unknown`

### Whatnstraints (as in Lakeflow)

```sql
CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW
CONSTRAINT valid_customer EXPECT (customer_id IS NOT NULL) ON VIOLATION DROP ROW
CONSTRAINT valid_product EXPECT (product_id IS NOT NULL) ON VIOLATION DROP ROW
CONSTRAINT valid_quantity EXPECT (quantity IS NOT NULL AND quantity <> 0) ON VIOLATION DROP ROW
CONSTRAINT valid_unit_price EXPECT (unit_price IS NOT NULL AND unit_price >= 0) ON VIOLATION DROP ROW
```

### MERGE Operation - Deduplication Pattern

**Issue:** Bronze contains duplicates (append-only)

**Solution:** MERGE in Silver (upsert)

```sql
MERGE INTO silver.orders_clean AS target
USING (
    SELECT DISTINCT *
    FROM bronze.orders_raw
    WHERE ingestion_ts > (
        SELECT MAX(load_ts) FROM silver.orders_clean
    )
) AS source
ON target.order_id = source.order_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *
```

### Example 2.1: Bronze → Silver Transformation (Orders)

**Objective:** Transform orders from Bronze to Silver with cleansing and validation.

In [0]:
# Bronze → Silver: Orders (Lakeflow model compliant)
bronze_orders = spark.table(f"{BRONZE_SCHEMA}.orders_raw")

# Transform & Validate - calculations as in Lakeflow silver_orders
silver_orders = (bronze_orders
    # Keys
    .withColumn("order_id", F.col("order_id"))
    .withColumn("customer_id", F.col("customer_id"))
    .withColumn("product_id", F.col("product_id"))
    .withColumn("store_id", F.col("store_id"))
    
    # Dates
    .withColumn("order_ts", F.to_timestamp(F.col("order_datetime")))
    .withColumn("order_date", F.to_date(F.col("order_datetime")))
    .withColumn("order_date_key", F.date_format(F.col("order_datetime"), "yyyyMMdd").cast("int"))
    
    # Original measures
    .withColumn("quantity", F.col("quantity").cast("int"))
    .withColumn("unit_price", F.col("unit_price").cast("double"))
    .withColumn("discount_percent", F.col("discount_percent").cast("int"))
    
    # Calculated measures (Lakeflow compliant)
    .withColumn("gross_amount", F.col("quantity") * F.col("unit_price"))
    .withColumn("discount_amount", F.col("quantity") * F.col("unit_price") * F.col("discount_percent") / 100.0)
    .withColumn("net_amount", 
        F.col("gross_amount") - F.col("discount_amount")
    )
    
    # Payment method (standardized)
    .withColumn("payment_method_code", F.coalesce(F.col("payment_method"), F.lit("Unknown")))
    .withColumn("source_system", F.col("source_system"))
    
    # Quality flags (Lakeflow compliant)
    .withColumn("is_return", 
        F.when((F.col("quantity") < 0) | (F.col("total_amount") < 0), 1).otherwise(0)
    )
    .withColumn("is_future_dated",
        F.when(F.col("order_date") > F.current_date(), 1).otherwise(0)
    )
    .withColumn("is_unknown_customer",
        F.when(F.col("customer_id").isNull() | (F.col("customer_id") == "CUST999999"), 1).otherwise(0)
    )
    .withColumn("is_unknown_product",
        F.when(F.col("product_id").isNull() | (F.col("product_id") == "PROD999999"), 1).otherwise(0)
    )
    
    # Validation - DROP invalid rows (as CONSTRAINT in Lakeflow)
    .filter(F.col("order_id").isNotNull())
    .filter(F.col("customer_id").isNotNull())
    .filter(F.col("product_id").isNotNull())
    .filter((F.col("quantity").isNotNull()) & (F.col("quantity") != 0))
    .filter((F.col("unit_price").isNotNull()) & (F.col("unit_price") >= 0))
    
    # Select final columns
    .select(
        "order_id", "customer_id", "product_id", "store_id",
        "order_ts", "order_date", "order_date_key",
        "quantity", "unit_price", "discount_percent",
        "gross_amount", "discount_amount", "net_amount",
        "payment_method_code", "source_system",
        "is_return", "is_future_dated", "is_unknown_customer", "is_unknown_product"
    )
)

# Write to Silver
silver_orders.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.orders_clean")

# Stats
bronze_count = bronze_orders.count()
silver_count = silver_orders.count()
dropped_count = bronze_count - silver_count

display({
    "bronze_records": bronze_count,
    "silver_records": silver_count,
    "dropped_by_constraints": dropped_count,
    "status": f"✅ Created {SILVER_SCHEMA}.orders_clean"
})

In [0]:
# Preview Silver Orders with new calculations
display(
    spark.table(f"{SILVER_SCHEMA}.orders_clean")
    .select("order_id", "order_date", "quantity", "unit_price", "discount_percent", 
            "gross_amount", "discount_amount", "net_amount", "is_return", "is_future_dated")
    .limit(10)
)

**Note about production:**

In a production environment, we would use MERGE operation for deduplication instead of simple overwrite.

## Section 3: SCD (Slowly Changing Dimensions)

**Section Objective:** Implementation of SCD Type 1 and Type 2 for tracking data changes.

### SCD Model in Lakeflow

In our Lakeflow pipeline, `silver_customers` uses **SCD Type 2** with columns:
- `__START_AT` - timestamp of record validity start
- `__END_AT` - timestamp of validity end (NULL = current record)

```sql
-- Lakeflow AUTO CDC for SCD2
CREATE FLOW silver_customers_scd2_flow
AS AUTO CDC INTO silver_customers
FROM STREAM bronze_customers
KEYS (customer_id)
SEQUENCE BY ingestion_ts
STORED AS SCD TYPE 2;
```

### SCD Types - Overview

| Type | Strategy | History | Use Case |
|------|----------|---------|----------|
| **Type 0** | No changes allowed | N/A | Reference data (countries) |
| **Type 1** | Overwrite | ❌ No | Current state only |
| **Type 2** | Add new row | ✅ Yes | Full history tracking |
| **Type 3** | Add new column | ⚠️ Limited | Previous value only |

---

### SCD Type 1 - Overwrite

**Strategy:** Overwrite old value with new (no history)

**Implementation:** Simple UPDATE/MERGE

**Pros:**
- ✅ Simple implementation
- ✅ No history bloat
- ✅ Always current values

**Whatns:**
- ❌ No historical tracking
- ❌ Can't analyze "as of date"
- ❌ Lose audit trail

**Use Cases:**
- Whatrrecting data entry errors
- Non-critical attributes (e.g., marketing preferences)

---

### SCD Type 2 - Historical Tracking (used in Lakeflow)

**Strategy:** Add new record for each change (full history)

**SCD Type 2 Whatlumns (Lakeflow format):**
- `__START_AT`: Validity start timestamp
- `__END_AT`: Validity end timestamp (NULL = current)

**Query current records:**
```sql
SELECT * FROM silver_customers WHERE __END_AT IS NULL
```

**Query historical (as of date):**
```sql
SELECT * FROM silver_customers 
WHERE '2024-01-15' >= __START_AT 
  AND ('2024-01-15' < __END_AT OR __END_AT IS NULL)
```

**Use Cases:**
- Customer dimensions (address, preferences)
- Product dimensions (price history)
- Whatmpliance & audit requirements

### Example 3.1: SCD Type 1 - Customers (Overwrite)

**Objective:** SCD Type 1 implementation - simple overwrite without history.

In [0]:
# SCD Type 1: Customers (Current State Only)
bronze_customers = spark.table(f"{BRONZE_SCHEMA}.customers_raw")

# Transform to Silver (SCD Type 1 - current state only)
customers_type1 = (bronze_customers
    .withColumn("customer_id", F.col("customer_id"))
    .withColumn("first_name", F.trim(F.col("first_name")))
    .withColumn("last_name", F.trim(F.col("last_name")))
    .withColumn("email", F.lower(F.trim(F.col("email"))))
    .withColumn("phone", F.col("phone"))
    .withColumn("city", F.initcap(F.trim(F.col("city"))))
    .withColumn("state", F.upper(F.trim(F.col("state"))))
    .withColumn("country", F.upper(F.trim(F.col("country"))))
    .withColumn("registration_date", F.to_date(F.col("registration_date")))
    .withColumn("customer_segment", F.col("customer_segment"))
    .withColumn("updated_at", F.current_timestamp())
    .select(
        "customer_id", "first_name", "last_name", "email", "phone",
        "city", "state", "country", "registration_date", "customer_segment",
        "updated_at"
    )
)

# Create/Replace table (Type 1 = overwrite)
customers_type1.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.customers_type1")

display({
    "status": f"✅ Created {SILVER_SCHEMA}.customers_type1",
    "records": customers_type1.count(),
    "note": "SCD Type 1: Always current state, no history"
})

**Created SCD Type 1 table:**

Table `customers_type1` always contains current state without history. Displaying sample data:

In [0]:
display(spark.table(f"{SILVER_SCHEMA}.customers_type1").limit(5))

### UPDATE Simulation - city change for customer_id=1

**BEFORE CHANGE:** Displaying current state of customer_id=1:

In [0]:
display(
    spark.sql(f"""
        SELECT customer_id, first_name, last_name, city, state, country, customer_segment
        FROM {SILVER_SCHEMA}.customers_type1 
        WHERE customer_id = 'CUST000001'
    """)
)

In [0]:
# UPDATE Simulation - city change for customer_id
from pyspark.sql.types import StructType, StructField, StringType

updates_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("country", StringType(), True),
    StructField("registration_date", StringType(), True),
    StructField("customer_segment", StringType(), True)
])

updates_data = [("CUST000001", "Jesse", "Hoffman", "jesse.hoffman@interia.pl", "+48 694 026 542", 
                 "Kraków", "MA", "POL", "2025-10-16", "Premium")]  # Changed city & country!
updates_df = spark.createDataFrame(updates_data, updates_schema)

In [0]:
# SCD Type 1: MERGE (Overwrite)
updates_df.createOrReplaceTempView("customer_updates")

spark.sql(f"""
    MERGE INTO {SILVER_SCHEMA}.customers_type1 AS target
    USING customer_updates AS source
    ON target.customer_id = source.customer_id
    WHEN MATCHED THEN UPDATE SET
        target.city = source.city,
        target.state = source.state,
        target.country = source.country,
        target.customer_segment = source.customer_segment,
        target.updated_at = current_timestamp()
    WHEN NOT MATCHED THEN INSERT (
        customer_id, first_name, last_name, email, phone,
        city, state, country, registration_date, customer_segment, updated_at
    ) VALUES (
        source.customer_id, source.first_name, source.last_name, source.email, source.phone,
        source.city, source.state, source.country, CAST(source.registration_date AS DATE), 
        source.customer_segment, current_timestamp()
    )
""")

**AFTER CHANGE (SCD Type 1 - overwrite):** Displaying updated state:

In [0]:
display(
    spark.sql(f"""
        SELECT customer_id, first_name, last_name, city, state, country, customer_segment
        FROM {SILVER_SCHEMA}.customers_type1 
        WHERE customer_id = 'CUST000001'
    """)
)

**⚠️  NOTE:** Change history has been LOST

**💡 Old value (Warsaw) was overwritten (Krakow)**

### Example 3.2: SCD Type 2 - Customers (Historical Tracking)

**Objective:** SCD Type 2 implementation - full change history tracking.

In [0]:
# SCD Type 2: Customers (Historical Tracking - Lakeflow format)
bronze_customers = spark.table(f"{BRONZE_SCHEMA}.customers_raw")

# Transform with SCD Type 2 columns (Lakeflow format: __START_AT, __END_AT)
customers_type2_initial = (bronze_customers
    .withColumn("customer_id", F.col("customer_id"))
    .withColumn("first_name", F.trim(F.col("first_name")))
    .withColumn("last_name", F.trim(F.col("last_name")))
    .withColumn("email", F.lower(F.trim(F.col("email"))))
    .withColumn("phone", F.col("phone"))
    .withColumn("city", F.initcap(F.trim(F.col("city"))))
    .withColumn("state", F.upper(F.trim(F.col("state"))))
    .withColumn("country", F.upper(F.trim(F.col("country"))))
    .withColumn("registration_date", F.to_date(F.col("registration_date")))
    .withColumn("customer_segment", F.col("customer_segment"))
    # SCD Type 2 columns (Lakeflow format)
    .withColumn("__START_AT", F.current_timestamp())
    .withColumn("__END_AT", F.lit(None).cast("timestamp"))  # NULL = current record
    .select(
        "customer_id", "first_name", "last_name", "email", "phone",
        "city", "state", "country", "registration_date", "customer_segment",
        "__START_AT", "__END_AT"
    )
)

# Create initial table
customers_type2_initial.write.format("delta").mode("overwrite").saveAsTable(f"{SILVER_SCHEMA}.customers_type2")

display({
    "status": f"✅ Created {SILVER_SCHEMA}.customers_type2",
    "records": customers_type2_initial.count(),
    "columns": "__START_AT, __END_AT (Lakeflow format)"
})

**Created SCD Type 2 table:**

Table `customers_type2` contains columns: `__START_AT`, `__END_AT` (Lakeflow format). Displaying sample data:

In [0]:
display(spark.table(f"{SILVER_SCHEMA}.customers_type2").limit(5))

### CHANGE Simulation - city change for customer_id=1

**BEFORE CHANGE:** Displaying current state of customer_id=1:

In [0]:
display(
    spark.sql(f"""
        SELECT customer_id, first_name, last_name, city, country, __START_AT, __END_AT
        FROM {SILVER_SCHEMA}.customers_type2 
        WHERE customer_id = 'CUST000001'
        ORDER BY __START_AT
    """)
)

In [0]:
# UPDATE Simulation for SCD Type 2 - same update as for Type 1
# We use the same customer_updates view (already created earlier)

**Step 1: Close old records** (set __END_AT):

In [0]:
# SCD Type 2: Step 1 - Close old records (set __END_AT)
spark.sql(f"""
    MERGE INTO {SILVER_SCHEMA}.customers_type2 AS target
    USING (
        SELECT DISTINCT u.customer_id
        FROM customer_updates u
        INNER JOIN {SILVER_SCHEMA}.customers_type2 t
            ON u.customer_id = t.customer_id
        WHERE t.__END_AT IS NULL
          AND (u.city != t.city OR u.country != t.country OR u.customer_segment != t.customer_segment)
    ) AS changed
    ON target.customer_id = changed.customer_id 
       AND target.__END_AT IS NULL
    WHEN MATCHED THEN UPDATE SET
        target.__END_AT = current_timestamp()
""")

**Step 2: Insert new versions** - adding new records with updated values:

In [0]:
# SCD Type 2: Step 2 - Insert new versions
spark.sql(f"""
    INSERT INTO {SILVER_SCHEMA}.customers_type2
    SELECT 
        u.customer_id,
        u.first_name,
        u.last_name,
        u.email,
        u.phone,
        u.city,
        u.state,
        u.country,
        CAST(u.registration_date AS DATE),
        u.customer_segment,
        current_timestamp() AS __START_AT,
        CAST(NULL AS TIMESTAMP) AS __END_AT
    FROM customer_updates u
    WHERE NOT EXISTS (
        SELECT 1 FROM {SILVER_SCHEMA}.customers_type2 existing
        WHERE existing.customer_id = u.customer_id
          AND existing.__END_AT IS NULL
          AND existing.city = u.city
          AND existing.country = u.country
          AND existing.customer_segment = u.customer_segment
    )
""")

**AFTER CHANGE (SCD Type 2 - historical tracking):**

History has been preserved! Displaying all versions of customer_id=1:

In [0]:
# Display change history for CUST000001
display(
    spark.sql(f"""
        SELECT 
            customer_id,
            first_name,
            city,
            country,
            customer_segment,
            __START_AT,
            __END_AT,
            CASE WHEN __END_AT IS NULL THEN 'CURRENT' ELSE 'HISTORICAL' END AS status
        FROM {SILVER_SCHEMA}.customers_type2 
        WHERE customer_id = 'CUST000001'
        ORDER BY __START_AT
    """)
)

**✅ History preserved! (Lakeflow format)**

We now have 2 records:
- **Record 1**: New York, USA (__END_AT = closing timestamp)
- **Record 2**: Krakow, POL (__END_AT = NULL = current record)

In Lakeflow, the same effect is achieved automatically via `AUTO CDC ... STORED AS SCD TYPE 2`.

### Example: Query 'as of date'

**Where did the customer live 1 month ago?** SCD Type 2 enables temporal queries:

In [0]:
one_month_ago = (datetime.now() - timedelta(days=30)).date()

In [0]:
# Query "as of date" - Lakeflow format
display(
    spark.sql(f"""
        SELECT 
            customer_id,
            first_name,
            city,
            country,
            __START_AT,
            __END_AT
        FROM {SILVER_SCHEMA}.customers_type2
        WHERE customer_id = 'CUST000001'
          AND '{one_month_ago}' >= __START_AT 
          AND ('{one_month_ago}' < __END_AT OR __END_AT IS NULL)
    """)
)

## Section 4: Gold Layer - Star Schema (Lakeflow compliant)

**Section Objective:** Implementation of Star Schema according to Lakeflow model.

### Gold Layer - Data Model (Lakeflow)

**Fact Table:**
- `fact_sales` - main fact table with transactional measures

**Dimension Tables:**
- `dim_customer` - customer dimension (snapshot from SCD2)
- `dim_product` - product dimension
- `dim_date` - time dimension
- `dim_store` - store dimension
- `dim_payment_method` - payment method dimension

### Star Schema Diagram

```
                    ┌─────────────────┐
                    │   dim_date      │
                    │ date_key (PK)   │
                    │ year, quarter   │
                    │ month, day      │
                    │ is_weekend      │
                    └────────┬────────┘
                             │
┌──────────────┐    ┌────────┴────────┐    ┌──────────────┐
│ dim_customer │    │   fact_sales    │    │ dim_product  │
│ customer_id  │────│ order_id (PK)   │────│ product_id   │
│ first_name   │    │ customer_id (FK)│    │ product_name │
│ last_name    │    │ product_id (FK) │    │ unit_cost    │
│ city, state  │    │ store_id (FK)   │    │ list_price   │
│ segment      │    │ date_key (FK)   │    │ is_active    │
└──────────────┘    │ payment_method  │    └──────────────┘
                    │ quantity        │
                    │ unit_price      │
                    │ gross_amount    │
                    │ discount_amount │
                    │ net_amount      │
                    │ is_return       │
                    └────────┬────────┘
                             │
              ┌──────────────┴──────────────┐
              │                             │
    ┌─────────┴─────────┐     ┌─────────────┴─────────┐
    │    dim_store      │     │ dim_payment_method    │
    │ store_id          │     │ payment_method_code   │
    │ store_code        │     │ payment_method_group  │
    └───────────────────┘     └───────────────────────┘
```

### Example 4.1: dim_customer

In [0]:
# Gold: dim_customer (snapshot from SCD2 - only current records)
dim_customer = spark.sql(f"""
    SELECT
        customer_id,
        first_name,
        last_name,
        email,
        phone,
        city,
        state,
        country,
        registration_date,
        customer_segment
    FROM {SILVER_SCHEMA}.customers_type2
    WHERE __END_AT IS NULL
""")

dim_customer.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.dim_customer")

display({
    "table": f"{GOLD_SCHEMA}.dim_customer",
    "records": dim_customer.count(),
    "status": "✅ Created"
})

**Created dim_customer table:**

In [0]:
display(spark.table(f"{GOLD_SCHEMA}.dim_customer").limit(5))

### Example 4.2: dim_product (Lakeflow compliant)

**Created dim_product table:**

In [0]:
# Gold: dim_product (Lakeflow silver_products compliant)
from pyspark.sql.types import (
    StringType, DoubleType, IntegerType, StructType, StructField
)

bronze_products = spark.table(f"{BRONZE_SCHEMA}.products_raw")

dim_product = (bronze_products
    .withColumn("unit_cost", F.col("unit_cost").cast("double"))
    .withColumn("list_price", F.col("list_price").cast("double"))
    .withColumn("weight_kg", F.col("weight_kg").cast("double"))
    .withColumn("is_active", 
        F.when(F.upper(F.col("status")).isin("ACTIVE", "AVAILABLE"), 1).otherwise(0)
    )
    .withColumn("is_unknown", F.lit(0))
    .select(
        "product_id", "product_name", "subcategory_code", "brand",
        "unit_cost", "list_price", "weight_kg", "status",
        "is_active", "is_unknown"
    )
)


unknown_schema = StructType([
    StructField("product_id", StringType(), False),
    StructField("product_name", StringType(), False),
    StructField("subcategory_code", StringType(), False),
    StructField("brand", StringType(), False),
    StructField("unit_cost", DoubleType(), True),
    StructField("list_price", DoubleType(), True),
    StructField("weight_kg", DoubleType(), True),
    StructField("status", StringType(), False),
    StructField("is_active", IntegerType(), False),
    StructField("is_unknown", IntegerType(), False)
])

unknown_product = spark.createDataFrame([
    ("UNKNOWN", "Unknown product", "UNKNOWN", "Unknown", None, None, None, "Unknown", 0, 1)
], schema=unknown_schema)

dim_product_final = dim_product.unionByName(unknown_product)

dim_product_final.write.format("delta").mode("overwrite").saveAsTable(
    f"{GOLD_SCHEMA}.dim_product"
)

display({
    "table": f"{GOLD_SCHEMA}.dim_product",
    "records": dim_product_final.count(),
    "status": "✅ Created (includes UNKNOWN product)"
})

In [0]:
display(spark.table(f"{GOLD_SCHEMA}.dim_product").limit(5))

### Example 4.3: dim_date

**Created dim_date table:**

In [0]:
# Gold: dim_date (generated from silver_orders as in Lakeflow)
silver_orders = spark.table(f"{SILVER_SCHEMA}.orders_clean")

dim_date = (silver_orders
    .select("order_date")
    .filter(F.col("order_date").isNotNull())  # Filter NULL dates
    .distinct()
    .withColumn("date_key", F.date_format(F.col("order_date"), "yyyyMMdd").cast("int"))
    .withColumn("date", F.col("order_date"))
    .withColumn("year", F.year(F.col("order_date")))
    .withColumn("quarter", F.quarter(F.col("order_date")))
    .withColumn("month", F.month(F.col("order_date")))
    .withColumn("day", F.dayofmonth(F.col("order_date")))
    .withColumn("day_of_week", F.date_format(F.col("order_date"), "E"))
    .withColumn("is_weekend", 
        F.when(F.date_format(F.col("order_date"), "E").isin("Sat", "Sun"), 1).otherwise(0)
    )
    .select("date_key", "date", "year", "quarter", "month", "day", "day_of_week", "is_weekend")
    .orderBy("date_key")
)

dim_date.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.dim_date")

display({
    "table": f"{GOLD_SCHEMA}.dim_date",
    "records": dim_date.count(),
    "status": "✅ Created"
})

In [0]:
display(spark.table(f"{GOLD_SCHEMA}.dim_date").limit(10))

### Example 4.5: dim_store and dim_payment_method

In [0]:
# Gold: dim_store (generated from silver_orders as in Lakeflow)
dim_store = (silver_orders
    .select("store_id")
    .distinct()
    .withColumn("store_code", F.col("store_id"))
)

dim_store.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.dim_store")

# Gold: dim_payment_method (generated from silver_orders as in Lakeflow)
dim_payment_method = (silver_orders
    .select("payment_method_code")
    .distinct()
    .withColumn("payment_method_group",
        F.when(F.col("payment_method_code").isin("Credit Card", "Debit Card"), "Card")
        .when(F.col("payment_method_code") == "Cash", "Cash")
        .when(F.col("payment_method_code") == "PayPal", "Digital wallet")
        .otherwise("Other")
    )
)

dim_payment_method.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.dim_payment_method")

display({
    "dim_store": {"records": dim_store.count(), "status": "✅"},
    "dim_payment_method": {"records": dim_payment_method.count(), "status": "✅"}
})

### Example 4.6: fact_sales (fact table)

In [0]:
# Gold: fact_sales (main fact table - Lakeflow compliant)
silver_orders = spark.table(f"{SILVER_SCHEMA}.orders_clean")

fact_sales = (silver_orders
    .select(
        # Dimension keys
        "order_id",
        "store_id",
        F.coalesce(F.col("customer_id"), F.lit("UNKNOWN")).alias("customer_id"),
        F.coalesce(F.col("product_id"), F.lit("UNKNOWN")).alias("product_id"),
        "payment_method_code",
        "order_date_key",
        "order_ts",
        
        # Measures
        "quantity",
        "unit_price",
        "discount_percent",
        "gross_amount",
        "discount_amount",
        "net_amount",
        
        # Flags
        "is_return",
        "is_future_dated",
        "is_unknown_customer",
        "is_unknown_product",
        
        # Lineage
        "source_system"
    )
)

fact_sales.write.format("delta").mode("overwrite").saveAsTable(f"{GOLD_SCHEMA}.fact_sales")

display({
    "table": f"{GOLD_SCHEMA}.fact_sales",
    "records": fact_sales.count(),
    "status": "✅ Created"
})

### Preview fact_sales with dimension joins

In [0]:
# Query Star Schema - sample sales report
display(
    spark.sql(f"""
        SELECT 
            d.date,
            d.day_of_week,
            c.first_name || ' ' || c.last_name AS customer_name,
            c.customer_segment,
            p.product_name,
            p.brand,
            pm.payment_method_group,
            f.quantity,
            f.gross_amount,
            f.discount_amount,
            f.net_amount
        FROM {GOLD_SCHEMA}.fact_sales f
        JOIN {GOLD_SCHEMA}.dim_date d ON f.order_date_key = d.date_key
        JOIN {GOLD_SCHEMA}.dim_customer c ON f.customer_id = c.customer_id
        JOIN {GOLD_SCHEMA}.dim_product p ON f.product_id = p.product_id
        JOIN {GOLD_SCHEMA}.dim_payment_method pm ON f.payment_method_code = pm.payment_method_code
        WHERE f.is_return = 0 AND f.is_future_dated = 0
        ORDER BY f.net_amount DESC
        LIMIT 10
    """)
)

## Section 5: Summary & Best Practices

### What was achieved?

✅ **1. Medallion Architecture Implementation (Lakeflow compliant)**

**🥉 Bronze Layer:**
- `bronze.customers_raw` - CSV with metadata (source_file_path, ingestion_ts, load_ts)
- `bronze.orders_raw` - JSON with source_system (batch/stream)
- `bronze.products_raw` - Parquet

**🥈 Silver Layer:**
- `silver.orders_clean` - with calculations (gross_amount, discount_amount, net_amount)
- `silver.orders_clean` - with flags (is_return, is_future_dated, is_unknown_*)
- `silver.customers_type1` - SCD Type 1 (current state)
- `silver.customers_type2` - SCD Type 2 (__START_AT, __END_AT)

**🥇 Gold Layer (Star Schema):**
- `fact_sales` - fact table with measures and flags
- `dim_customer` - snapshot from SCD2
- `dim_product` - with is_active, is_unknown
- `dim_date` - date_key, year, quarter, month, is_weekend
- `dim_store` - store_id, store_code
- `dim_payment_method` - payment_method_group (Card/Cash/Digital wallet)

✅ **2. SCD (Slowly Changing Dimensions)**
- **Type 1**: Overwrite (customers_type1)
- **Type 2**: Historical tracking with __START_AT/__END_AT (customers_type2, Lakeflow format)

✅ **3. Data Quality (Lakeflow CONSTRAINTS compliant)**
- valid_order_id: DROP ROW if NULL
- valid_customer: DROP ROW if NULL
- valid_product: DROP ROW if NULL
- valid_quantity: DROP ROW if NULL or 0
- valid_unit_price: DROP ROW if NULL or < 0

### Data Model - Whatmparison with Lakeflow

| Lakeflow (Streaming) | Notebook (Batch) | Status |
|---------------------|------------------|--------|
| `bronze_customers` (STREAMING TABLE) | `bronze.customers_raw` | ✅ |
| `bronze_orders` (STREAMING TABLE + FLOW) | `bronze.orders_raw` | ✅ |
| `bronze_products` (MATERIALIZED VIEW) | `bronze.products_raw` | ✅ |
| `silver_customers` (SCD2 AUTO CDC) | `silver.customers_type2` | ✅ |
| `silver_orders` (CONSTRAINTS) | `silver.orders_clean` | ✅ |
| `fact_sales` (STREAMING TABLE) | `gold.fact_sales` | ✅ |
| `dim_customer` (MV) | `gold.dim_customer` | ✅ |
| `dim_product` (MV) | `gold.dim_product` | ✅ |
| `dim_date` (MV) | `gold.dim_date` | ✅ |
| `dim_store` (MV) | `gold.dim_store` | ✅ |
| `dim_payment_method` (MV) | `gold.dim_payment_method` | ✅ |

### Key Takeaways

💡 **1. Lakeflow Streaming vs Batch**
```
Lakeflow: CREATE OR REFRESH STREAMING TABLE + FLOW
Notebook: spark.read → transform → write (batch)

Both methods create an identical data model!
```

💡 **2. Calculations in Silver (not Bronze)**
```python
# Silver orders - calculated measures
gross_amount = quantity * unit_price
discount_amount = gross_amount * discount_percent / 100
net_amount = gross_amount - discount_amount
```

💡 **3. SCD Type 2 - Lakeflow format**
```python
# Lakeflow: AUTO CDC ... STORED AS SCD TYPE 2
# Notebook: Manual MERGE with __START_AT, __END_AT

# Query current: WHERE __END_AT IS NULL
# Query historical: WHERE date BETWEEN __START_AT AND __END_AT
```

💡 **4. Star Schema Query Pattern**
```sql
SELECT f.*, d.year, c.customer_segment, p.product_name
FROM fact_sales f
JOIN dim_date d ON f.order_date_key = d.date_key
JOIN dim_customer c ON f.customer_id = c.customer_id
JOIN dim_product p ON f.product_id = p.product_id
```

**🛠️ Lakeflow Pipeline:**
- Review SQL files in `/Lakeflow/` for production-ready streaming implementation

---

**Whatngratulations!** 🎉 
You have completed the Medallion Architecture implementation following the Lakeflow model!
You are now ready to build production-grade data lakehouse pipelines!

## Section 6: Resource Cleanup

**Note:** This section is optional. Run only if you want to delete all data created in this notebook.

### Option 1: Check created resources (recommended)

**Data is preserved for further use**

To delete all tables, run the next cell in the optional section.

### Option 2: Delete all resources (only if you really want to)

**WARNING:** This will delete all Silver and Gold tables created in this notebook!

In [0]:
# Option 2: Delete all resources (ONLY IF YOU ARE SURE!)

# ⚠️  WARNING: Uncomment the code below only if you want to delete everything!

"""
print("=== 🗑️  DELETING MEDALLION RESOURCES ===\n")

# List of tables to delete (Star Schema)
tables_to_drop = [
    # Silver
    f"{SILVER_SCHEMA}.orders_clean",
    f"{SILVER_SCHEMA}.customers_type1",
    f"{SILVER_SCHEMA}.customers_type2",
    # Gold - Star Schema
    f"{GOLD_SCHEMA}.fact_sales",
    f"{GOLD_SCHEMA}.dim_customer",
    f"{GOLD_SCHEMA}.dim_product",
    f"{GOLD_SCHEMA}.dim_date",
    f"{GOLD_SCHEMA}.dim_store",
    f"{GOLD_SCHEMA}.dim_payment_method"
]

print("Deleting tables...\n")
for table in tables_to_drop:
    full_table = f"{CATALOG}.{table}"
    try:
        spark.sql(f"DROP TABLE IF EXISTS {full_table}")
        print(f"  ✓ Deleted: {table}")
    except Exception as e:
        print(f"  ⚠️  Error with {table}: {e}")

print("\n✅ Cleanup completed!")
"""

display({
    "status": "⚠️ CLEANUP CODE IS COMMENTED OUT",
    "info": "Uncomment the code above only if you want to delete all resources",
    "recommendation": "Keep the data for subsequent notebooks and workshops!",
    "next": "04_optimization_best_practices.ipynb"
})